In [2]:
!pip install --upgrade boto3 -i https://pypi.tuna.tsinghua.edu.cn/simple/
!pip install --upgrade sagemaker -i https://pypi.tuna.tsinghua.edu.cn/simple/
!pip install huggingface-hub -Uqq -i https://pypi.tuna.tsinghua.edu.cn/simple/

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/, https://pip.repos.neuron.amazonaws.com


In [3]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_model_path = Path("./LLM_chatglm_model")
local_model_path.mkdir(exist_ok=True)
model_name = "THUDM/chatglm-6b"
commit_hash = "f83182484538e663a03d3f73647f10f89878f438"

In [ ]:
#执行这段，因为网络原因，可能会多次失败，需要反复执行
while True:
    try:
        snapshot_download(repo_id=model_name, revision=commit_hash, cache_dir=local_model_path)
        break
    except Exception:
        pass

In [4]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

In [ ]:
s3_model_prefix = "LLM-RAG/workshop/LLM_chatglm_model"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/LLM_chatglm_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

In [ ]:

#上传模型至S3
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

In [15]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)

if "cn-" in region_name:
    with open('./code/requirements.txt', 'r') as original: data = original.read()
    with open('./code/requirements.txt', 'w') as modified: modified.write("-i https://pypi.tuna.tsinghua.edu.cn/simple\n" + data)

!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'llm_chinese')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'llm_chinese')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

model_name = None
entry_point = 'inference.py'
framework_version = '1.13.1'
py_version = 'py39'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
}

from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(
    name = model_name,
    model_data = model_data,
    entry_point = entry_point,
    source_dir = './code',
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

endpoint_name = 'pytorch-inference-llm-v1'
#instance_type = 'ml.p3.2xlarge'
instance_type='ml.g4dn.2xlarge' 

instance_count = 1

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)

arn:aws-cn:iam::507392672631:role/NotebookStack-SmartSearchNotebookRole6F6BB12B-XOVHSG6091PM
sagemaker-cn-north-1-507392672631
dummy
upload: ./model.tar.gz to s3://sagemaker-cn-north-1-507392672631/llm_chinese/assets/model.tar.gz
--------------!

### 测试


In [ ]:
#休眠2分钟,确保模型可以完全加载
import time
time.sleep(120)

In [16]:
inputs= {
    "ask": "你好!"

}

response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "晚上睡不着应该怎么办"

}

response = predictor.predict(inputs)
print(response["answer"])

你好👋！很高兴见到你，欢迎问我任何问题。
晚上睡不着可能会让人感到焦虑和不安，但以下是一些可能有用的建议：

1. 保持放松：尝试进行深呼吸、渐进性肌肉松弛或冥想等放松技巧，帮助放松身体和头脑。

2. 避免使用电子设备：电子设备的蓝光可能会干扰睡眠，因此建议在睡前几个小时停止使用电子设备。

3. 创造一个舒适的睡眠环境：保持房间安静、凉爽、黑暗和舒适，有助于更容易入睡。

4. 避免饮食和饮料的刺激：避免饮用咖啡、茶、可乐等刺激性饮料，以及吃辛辣、油腻或难以消化的食物。

5. 尝试一些放松的活动：例如读书、听轻柔的音乐、泡个热水澡或进行瑜伽等，帮助放松身心。

6. 如果一直睡不着，不要强迫自己入睡：如果在床上躺了20分钟还是无法入睡，那么不要强迫自己入睡，可以起床做些放松的活动，直到感到困倦为止。

如果以上方法无法缓解失眠问题，建议咨询医生或专业的睡眠专家。


In [13]:
inputs= {
    "ask": "列出一些年夜饭好意头的菜肴以及其寓意!"

}

response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "帮我写一篇人工智能课程的教案，1000字"

}

response = predictor.predict(inputs)
print(response["answer"])

年夜饭是中国传统文化中非常重要的一部分，通常在农历新年除夕夜与家人共聚一堂，享受丰盛的饭菜。以下是一些常见的年夜饭好意头的菜肴以及其寓意：

1. 鱼：年夜饭中必不可少的一道菜是鱼，因为“鱼”和“余”谐音，寓意着年年有余，表示家中财源广进。

2. 鸡：鸡肉象征着“团圆”，因为鸡在中文中也有“家”的意思，所以鸡也象征着家庭团圆。

3. 红烧肉：红烧肉是一道非常传统的中国菜肴，其寓意为“红红火火”。

4. 年糕：年糕是中国传统的美食，因为“糕”和“高”谐音，寓意着新年里生活步步高升，也是年夜饭中常见的食物。

5. 葱姜蒜炒虾：虾的寓意为“生意兴隆”，而葱姜蒜则象征着财源广进。

6. 红烧肉炖豆腐：红烧肉和豆腐都是传统的中国美食，寓意为“年年有余”和“团圆”。

7. 饺子：饺子在中国传统文化中象征着团圆和家庭和睦，也在年夜饭中非常常见。

8. 汤圆：汤圆是中国传统的美食，通常和元宵节相关。汤圆的寓意为“团圆”，也是年夜饭中常见的食物。

以上是一些常见的年夜饭好意头的菜肴以及其寓意，当然，不同地区和不同家庭的年夜饭菜肴也非常丰富多彩，可以根据自己的口味和喜好选择搭配。
教案题目：人工智能基础课程教学与实践

教学目标：

1. 掌握人工智能的基本概念和原理，了解人工智能的应用和发展趋势。

2. 学会使用Python编程语言进行人工智能实践，掌握常见的人工智能算法和工具。

3. 培养学生的实践能力和创新意识，提高学生的综合素质。

教学内容：

1. 人工智能的基本概念和原理

2. Python编程语言的基本语法和常用库

3. 常见的人工智能算法和工具

教学方式：

1. 课堂讲解和案例分析

2. 实践操作和项目开发

3. 小组讨论和创新思维训练

教学步骤：

1. 介绍人工智能的基本概念和原理，让学生了解人工智能的定义、应用领域和发展趋势。

2. 介绍Python编程语言的基本语法和常用库，让学生了解Python编程语言的特点和优势。

3. 介绍常见的人工智能算法和工具，让学生了解人工智能的具体应用和实现方法。

4. 实践操作和项目开发，让学生通过实践操作和项目开发，掌握人工智能的实践技能和创新意识。

5. 小组讨论和创新思维训练，让学生通过小组讨论和创新思维训练，提高学生的综合素质和创新能力。

教学评价：

1. 课堂讲解和案例分

In [ ]:
inputs= {
    "ask": "怎么修改huggingface transformers的model cache位置"

}

response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "用python3写出快速排序代码"

}

response = predictor.predict(inputs)
print(response["answer"])

In [ ]:
#我们来查看一下刚部署的这个模型对应的SageMaker inference endpoint
sagemaker_endpoint_name = predictor.endpoint_name
sagemaker_endpoint_name

利用已经在SageMaker real time inference endpoint部署的LLM模型来模拟单轮对话和多轮对话。

下面的代码建议在SageMaker Notebook上来运行。

In [18]:
import json
import boto3

client = boto3.client('runtime.sagemaker')
sagemaker_endpoint_name='pytorch-inference-llm-v1'
def query_endpoint_with_json_payload(encoded_json):
    response = client.invoke_endpoint(EndpointName=sagemaker_endpoint_name, ContentType='application/json', Body=encoded_json)
    return response

def parse_response_texts(query_response):
    model_predictions = json.loads(query_response['Body'].read())
    generated_text = model_predictions["answer"]
    return generated_text


先简单测试一下针对单个问题的回答

In [19]:
payload = {"ask": "信息抽取：\n2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案："}
query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
generated_texts = parse_response_texts(query_response)
print(generated_texts)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (424) from model with message "{
  "code":424,
  "message":"prediction failure",
  "error":"'inputs'"
}". See https://cn-north-1.console.aws.amazon.com/cloudwatch/home?region=cn-north-1#logEventViewer:group=/aws/sagemaker/Endpoints/pytorch-inference-llm-v1 in account 507392672631 for more information.

单轮对话：每个问题/query都是独立的，问题之间没有关联性。

In [ ]:
#1.首先需要一个简单的开场拍。
print("用户：你好！\nAI：您好!我是AI。我可以回答您的问题、写文章、写作业、翻译，对于一些法律等领域的问题我也可以给你提供信息。")

#2.在同一个session中持续对话，但是每次对话之间没有什么关联。
while True:
    command = input("用户：")
    if command == 'quit':
        break;
    
    #print(command)
    payload = {"ask": "\n用户："+ command + "\n"}
    #print(payload["ask"])
    query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
    generated_texts = "AI：" + parse_response_texts(query_response)
    print(generated_texts)

多轮对话模拟：我们这里来测试一下多轮对话能力。

In [ ]:
#1.首先需要开场拍来引导AI，其实就是给它一个上下文来启动所谓的对话session。
payload = {"ask": "用户：你好！\nAI：您好!我是AI。我可以回答您的问题、写文章、写作业、翻译，对于一些法律等领域的问题我也可以给你提供信息。"}
print(payload["ask"])
generated_texts = ""

#在这里简单模拟多轮对话时，发送给SageMaker endpoint的payload会越来越大，这里对payload大致做一个限制。
session_len = 10 * 1024 * 1024 

#2.在同一个session中持续对话，为了有多轮对话的效果，把每一轮的信息(问题-回答对)都带上来告诉LLM之前的上下文。
while True:
    command = input("用户：")
    if command == 'quit':
        break;
    
    #print(command)
    whole_context = payload["ask"] + generated_texts + "\n用户："+ command + "\n"
    payload = {"ask": whole_context}
    if len(whole_context) > session_len:
        print("上下文信息太多了，当前对话session要退出了！")
        break;
    #print(payload["ask"])
    query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
    generated_texts = "AI：" + parse_response_texts(query_response)
    print(generated_texts)

### 删除SageMaker  Endpoint
删除推理服务

In [ ]:
predictor.delete_endpoint()